### **First Stage Sentence Retrieval using SBERT**

In [30]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


In [31]:
import json

# Load datasets
path = 'data/'

with open(path+"train-claims.json", "r") as f:
    train_claims = json.load(f)

train_df = pd.DataFrame(train_claims).transpose()

with open(path+"dev-claims.json", "r") as f:
    dev_claims = json.load(f)
dev_df = pd.DataFrame(dev_claims).transpose()

with open(path+"evidence.json", "r") as f:
    evidence = json.load(f)
evidence_df = pd.DataFrame(list(evidence.items()), columns=["key", "value"])

In [32]:
# Load model
model1 = SentenceTransformer('sbert_model')
model2 = SentenceTransformer('sbert_model2')
model3 = SentenceTransformer('sbert_model3')

In [33]:
embeddings1 = model1.encode(evidence_df['value'].tolist(), show_progress_bar=True, device='cuda')
embeddings2 = model2.encode(evidence_df['value'].tolist(), show_progress_bar=True, device='cuda')
embeddings3 = model3.encode(evidence_df['value'].tolist(), show_progress_bar=True, device='cuda')

Batches:   0%|          | 0/37776 [00:00<?, ?it/s]

Batches:   0%|          | 0/37776 [00:00<?, ?it/s]

Batches:   0%|          | 0/37776 [00:00<?, ?it/s]

In [34]:
claim_embeddings1 = model1.encode(dev_df['claim_text'].tolist(), show_progress_bar=True, device='cuda')
claim_embeddings2 = model2.encode(dev_df['claim_text'].tolist(), show_progress_bar=True, device='cuda')
claim_embeddings3 = model3.encode(dev_df['claim_text'].tolist(), show_progress_bar=True, device='cuda')

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
claim_combined = (claim_embeddings1 + claim_embeddings2 + claim_embeddings3) / 3
evidence_combined = (embeddings1 + embeddings2 + embeddings3) / 3


RuntimeError: mat1 and mat2 shapes cannot be multiplied (154x768 and 384x768)

In [37]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(claim_combined , evidence_combined)

In [38]:
import numpy as np
# Get top 10000 evidence for each claim
top_k = 100
top_indices = np.argsort(-similarity_matrix, axis=1)[:, :top_k]  # sort descending, get top 100 indices

In [39]:
# Create new column with top evidence texts for each claim
dev_df["top_{top_k}_evidence"] = [
    evidence_df.iloc[indices]["key"].tolist() for indices in top_indices
]

In [41]:
total_matches = 0
n = 0 
for i in range(len(dev_df)):
    k = 0
    evidence_list = dev_df["evidences"].iloc[i]
    top_100_list = dev_df["top_{top_k}_evidence"].iloc[i]

    for ev in evidence_list:
        if ev in top_100_list:
            k += 1
    print(f"Found {k} matches for gold standard {len(evidence_list)}")
    n += len(evidence_list)
    total_matches += k

print(f"Total matches: {total_matches}/{n}")

Found 2 matches for gold standard 2
Found 0 matches for gold standard 5
Found 2 matches for gold standard 2
Found 4 matches for gold standard 5
Found 4 matches for gold standard 4
Found 2 matches for gold standard 5
Found 2 matches for gold standard 5
Found 1 matches for gold standard 2
Found 3 matches for gold standard 4
Found 0 matches for gold standard 3
Found 1 matches for gold standard 1
Found 2 matches for gold standard 2
Found 3 matches for gold standard 3
Found 2 matches for gold standard 5
Found 5 matches for gold standard 5
Found 1 matches for gold standard 5
Found 0 matches for gold standard 1
Found 1 matches for gold standard 4
Found 5 matches for gold standard 5
Found 3 matches for gold standard 5
Found 2 matches for gold standard 2
Found 0 matches for gold standard 2
Found 4 matches for gold standard 5
Found 2 matches for gold standard 3
Found 3 matches for gold standard 5
Found 0 matches for gold standard 1
Found 0 matches for gold standard 3
Found 1 matches for gold sta